
<img src="img/Logotipo_Colegio_Administradores.png" alt="Logo" width="600px" height="auto" />


# Introduction

In this project, we will collaborate with diverse teams, including UX and Full-Stack, to create a web app for the Colegio de Administradores de Valencia in Valencia. The primary objective is to facilitate seamless information sharing, fostering connections between administrators and neighbors for the benefit of the entire community. They depend on us to address specific needs, and they also welcome additional improvements that we will consider along the way. 

# Table of Contents


1. [Chapter 1 - Defining the Problem](#ch3)

2. [Chapter 2 - Designing the functions and testing](#ch4)

3. [Chapter 3 - Creating the app in Flask](#ch5)

4. [Chapter 4 - API Testing](#ch6)

<a id="ch3"></a>
# 1. Defining the Problem



As the data science team, our assigned task is to create a calculator. This tool will be used to estimate the amount of money neighbors will pay in the next years for future expenses.

We have developed two calculators with specific purposes: one for determining the required savings to reach a financial goal and the other for calculating the increased monthly payment needed to cover particular expenses. These calculators provide a comprehensive solution for effective financial management, offering precision and efficiency in planning and achieving economic objectives.

<a id="ch4"></a>
# 2. Designing the functions and testing

### 2.1 Import Libraries 

In [1]:
from flask import Flask, request, jsonify
import math
from flask_cors import CORS

### 2.2 Building the functions

Function to calculate total dues based on multiple periods and variables:

In [ ]:
def calculo_prev(*tuplas, n_propie, cuota):
    # Sorting tuples by the first element of each
    tuplas = sorted(tuplas, key=lambda x: x[0])

    # Creating a list of lists to separate elements of the tuples
    variables = [[] for _ in range(len(tuplas))]

    # Iterating over elements of the tuples and distributing them into 'variables'
    for i in range(len(tuplas[0])):
        valores = [t[i] for t in tuplas]
        for j, valor in enumerate(valores):
            variables[j].append(valor)

    # Calculating total months and periods from the variables
    total_meses = 0
    periodos = []
    for var in variables:
        periodos.append(var[0])
        if var[0] > total_meses:
            total_meses = var[0]

    # Calculating expense per month for each period
    gasto_x_mes = []
    suple_var = 0
    for i in range(total_meses):
        for var in variables:
            if i < var[0]:
                spl = round((var[1]/var[0])/n_propie, 2)
                suple_var += spl
        gasto_x_mes.append(suple_var)
        suple_var = 0

    # Calculating the total dues for each period
    cuotas_total = []
    for i in periodos:
        cuotas_total.append(math.ceil(gasto_x_mes[i -1] + cuota))

    # Returning the list of total dues
    return cuotas_total

Function to calculate monthly due based on projected expenses and savings:

In [ ]:

def suma():
    # Definition of initial variables
    cuota_actual = 40  # Current due
    n_propie = 10  # Number of owners
    ahorro = 10000  # Total savings amount
    fondos_iniciales = 1000  # Initial funds
    anios = 5  # Number of years for the calculation

    # Calculating total time in months and the actual expense
    tiempo = anios * 12  # Total months
    gasto_real = ahorro - fondos_iniciales  # Expense after deducting initial funds

    # Calculating supplement per owner and the resulting due
    suplemento = (gasto_real/tiempo)/n_propie
    resultado = cuota_actual + suplemento

    # Returning the resulting monthly due
    return resultado

<a id="ch4"></a>
# 3. Creating the app in Flask

### 3.1 Deployed the calculator's API

In [ ]:
from flask import Flask, request, jsonify
import math
from flask_cors import CORS

# Define a function to calculate future expenses.
def calculo_prev(*tuplas, n_propie, cuota):
    # Sort tuples based on the first element of each tuple.
    tuplas = sorted(tuplas, key=lambda x: x[0])

    # Create a list of lists to store variables.
    variables = [[] for _ in range(len(tuplas))]

    # Iterate over elements in tuples and organize data.
    for i in range(len(tuplas[0])):
        valores = [t[i] for t in tuplas]
        for j, valor in enumerate(valores):
            variables[j].append(valor)

    # Calculate the maximum number of months and prepare for calculations.
    total_meses = 0
    suple_var = 0
    periodos = []
    for var in variables:
        periodos.append(var[0])
        if var[0] > total_meses:
            total_meses = var[0]

    # Calculate monthly expenses.
    gasto_x_mes = []
    for i in range(total_meses):
        for var in variables:
            if i < var[0]:
                spl = round((var[1]/var[0])/n_propie, 2)
                suple_var += spl
        gasto_x_mes.append(suple_var)
        suple_var = 0

    # Calculate total quotas.
    cuotas_total = []
    for i in periodos:
        cuotas_total.append(math.ceil(gasto_x_mes[i - 1] + cuota))

    return cuotas_total

# Initialize Flask app and enable Cross-Origin Resource Sharing (CORS).
app = Flask(__name__)
CORS(app)

# Define a route for the root URL to return a welcome message.
@app.route('/', methods=['GET'])
def Hello():
    return 'Hello to the DS API'

# Define a route for calculating the new quota.
@app.route('/calculadora', methods=['POST'])
def suma():
    # Get data from the request's JSON body.
    data = request.json

    # Extract and calculate necessary variables.
    cuota_actual = data['cuota']
    n_propie = data['n_propietarios']
    ahorro = data['ahorro']
    fondos_iniciales = data['fondos']
    anios = data['anios']

    tiempo = anios * 12
    gasto_real = ahorro - fondos_iniciales 
    suplemento = (gasto_real/tiempo)/n_propie
    resultado = cuota_actual + ((gasto_real/tiempo)/n_propie)

    # Return the calculated new quota in JSON format.
    return jsonify({'Nueva cuota': resultado})

# Define a route for calculating average future quotas.
@app.route('/calculadora_previ', methods=['POST'])
def calculadora_prevision():
    # Get data from the request's JSON body.
    data = request.json

    # Process input data and call the calculation function.
    tuplas = [(item['meses'], item['monto']) for item in data.get('tuplas', [])]
    n_propie = int(data['n_propietarios'])
    cuota = int(data['cuota'])

    output = calculo_prev(*tuplas, n_propie=n_propie, cuota=cuota)
    output = math.ceil(sum(output)/len(output))

    # Return the average of new quotas in JSON format.
    return jsonify({'Nueva/s cuota/s': output})

# Run the app in debug mode on port 8000.
if __name__ == "__main__":
    app.run(debug=True, port=8000)


### 3.2 ChatGPT API's prototype 

This script is a command-line based chatbot application that interacts with the OpenAI ChatGPT API. It allows users to have a conversation with the AI model in a loop. The script provides functionalities to start a new conversation or exit the application. User inputs and AI responses are managed in a conversational history. The rich library is used for displaying styled text and tables, while typer is employed for handling user input and command-line interactions. The key features include handling custom commands, maintaining session context, and dynamically interacting with the OpenAI API to generate responses.

In [ ]:
import openai  # pip install openai
import typer  # pip install "typer[all]"
from rich import print  # pip install rich
from rich.table import Table

def main():
    # Set the OpenAI API key. Replace "TU_API_KEY" with your actual API key.
    openai.api_key = "TU_API_KEY creada en https://platform.openai.com"

    # Print a styled header using 'rich'.
    print("💬 [bold green]ChatGPT API en Python[/bold green]")

    # Create and display a table with commands and their descriptions.
    table = Table("Comando", "Descripción")
    table.add_row("exit", "Salir de la aplicación")
    table.add_row("new", "Crear una nueva conversación")
    print(table)

    # Initialize the assistant's context.
    context = {"role": "system", "content": "Eres un asistente muy útil."}
    messages = [context]

    # Main loop for user interaction.
    while True:
        content = __prompt()  # Get user input.

        # Handle 'new' command: reset conversation.
        if content == "new":
            print("🆕 Nueva conversación creada")
            messages = [context]
            content = __prompt()

        # Append user message to conversation history.
        messages.append({"role": "user", "content": content})

        # Send the conversation history to OpenAI and get a response.
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages)
        response_content = response.choices[0].message.content

        # Append the response to conversation history and print it.
        messages.append({"role": "assistant", "content": response_content})
        print(f"[bold green]> [/bold green] [green]{response_content}[/green]")

# Function to prompt the user for input.
def __prompt() -> str:
    prompt = typer.prompt("\n¿Sobre qué quieres hablar? ")

    # Handle 'exit' command: confirm before exiting.
    if prompt == "exit":
        exit = typer.confirm("✋ ¿Estás seguro?")
        if exit:
            print("👋 ¡Hasta luego!")
            raise typer.Abort()

        return __prompt()

    return prompt

# Entry point for the script.
if __name__ == "__main__":
    typer.run(main)


<a id="ch4"></a>
## 4. API Testing

### 4.1 Savings calculator

url = https://prueba-dev-xcjh.1.ie-1.fl0.io/calculadora


DESIRED SAVINGS:

```
{
    "cuota": 50,
    "n_propietarios": 15,
    "ahorro": 8000,
    "fondos": 1500,
    "anios": 5
}
```

The calculator provides us with the new quota value required to achieve the desired savings within the specified time.

It will return something similar to this:

```
{
    "Nueva cuota":57.2
}
```


### 4.2 Preventive cost calculator


url = https://fl0-dev-tdfm.1.us-1.fl0.io//calculadora_previ


FUTURE EXPENSES:

```
{
  "tuplas": [
    {"años": 1, "monto": 1000},
    {"años": 3, "monto": 3000},
    {"años": 5, "monto": 5000}
  ],
  "n_propietarios": 6,
  "cuota": 50
}
```


It returns the amount that each neighbor has to pay per month to cover each anticipated expense until the year of payment arrives.

It will return something similar to this:

```
{
    "Nueva/s cuota/s": 45.84
}
```


## Credits

Felipe Reus    |    Eliana Pinto    |    Diego Mateos    |    Alejandro Echevarria